<a href="https://colab.research.google.com/github/aleks-haksly/VIZRO/blob/main/demos/heatmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install vizro -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.0/857.0 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.8/500.8 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 16.5 MB/s eta 0:00:00


In [3]:
from sqlalchemy import text
from sqlalchemy import create_engine
import pandas as pd
from google.colab import userdata


engine = create_engine(userdata.get("supabase"), client_encoding='utf8' )

def select(sql):
    sql = text(sql)
    return pd.read_sql(sql, engine)

In [4]:
df = select("SELECT * FROM vizro.yandex_data_agg")

In [45]:
df_2 = select("SELECT *, date_part('h', ds) as hour, count - lag(count, 1) OVER (partition by platform, weekday, date_part('h', ds) order by ds) as wow_diff FROM vizro.yandex_data_agg")

In [48]:
df_2['date'] = df_2["ds"].dt.date

In [5]:
df['hour'] = df["ds"].dt.hour
df['date'] = df["ds"].dt.date

In [59]:
df_2.sort_values(by=['platform','ds'])

,ds,platform,weekday,weekend,count,hour,wow_diff,date
144,2021-09-01 00:00:00,desktop,3,False,360,0.0,NaN,2021-09-01
147,2021-09-01 01:00:00,desktop,3,False,191,1.0,NaN,2021-09-01
150,2021-09-01 02:00:00,desktop,3,False,152,2.0,NaN,2021-09-01
153,2021-09-01 03:00:00,desktop,3,False,139,3.0,NaN,2021-09-01
156,2021-09-01 04:00:00,desktop,3,False,168,4.0,NaN,2021-09-01
...,...,...,...,...,...,...,...,...
635,2021-09-21 19:00:00,touch,2,False,2339,19.0,120.0,2021-09-21
638,2021-09-21 20:00:00,touch,2,False,2581,20.0,35.0,2021-09-21
641,2021-09-21 21:00:00,touch,2,False,2418,21.0,-375.0,2021-09-21
644,2021-09-21 22:00:00,touch,2,False,1954,22.0,-120.0,2021-09-21


In [36]:
df.groupby(["platform", "weekday", 'hour'])["count"].sum()

platform  weekday  hour
desktop   1        0       1009
                   1        549
                   2        378
                   3        337
                   4        464
                           ... 
touch     7        19      6545
                   20      7182
                   21      7031
                   22      5836
                   23      3894
Name: count, Length: 336, dtype: int64

In [34]:
df.groupby(["platform", 'hour']).shift(1)

,ds,weekday,weekend,count,date
0,NaT,NaN,NaN,NaN,NaN
1,NaT,NaN,NaN,NaN,NaN
2,NaT,NaN,NaN,NaN,NaN
3,NaT,NaN,NaN,NaN,NaN
4,2021-09-04 01:00:00,6.0,True,167.0,2021-09-04
...,...,...,...,...,...
1003,2021-09-03 04:00:00,5.0,False,506.0,2021-09-03
1004,2021-09-01 03:00:00,3.0,False,715.0,2021-09-01
1005,2021-09-12 18:00:00,7.0,True,1299.0,2021-09-12
1006,2021-09-17 03:00:00,5.0,False,363.0,2021-09-17


In [51]:
import vizro.models as vm
import vizro.plotly.express as px
from vizro import Vizro

tips = px.data.tips()

fig = px.density_heatmap(
    df, x="date", y="hour", z="count", histfunc="sum", text_auto=True, nbinsy=24, nbinsx=21,
)

page = vm.Page(title="My page", components=[vm.Graph(figure=fig)])
dashboard = vm.Dashboard(pages=[page])
Vizro().build(dashboard).run()

ValueError: Page with id=My page already exists. Page id is automatically set to the same as the page title. If you have multiple pages with the same title then you must assign a unique id.

In [74]:
fig = px.density_heatmap(
    df_2[df.date >= df.date.max() - pd.Timedelta(7, 'days')], x="date", y="hour", z="count", histfunc="sum", text_auto=True, nbinsy=24, nbinsx=7, facet_col='platform')
fig

In [72]:
fig = px.density_heatmap(
    df_2[df.date >= df.date.max() - pd.Timedelta(7, 'days')], x="date", y="hour", z="wow_diff", histfunc="sum", text_auto=True, nbinsy=24, nbinsx=7, color_continuous_scale=px.colors.diverging.RdBu, color_continuous_midpoint=0, facet_col='platform'
)
fig

,ds,platform,weekday,weekend,count,hour,wow_diff,date
3,2021-09-06 01:00:00,desktop,1,False,167,1.0,NaN,2021-09-06
6,2021-09-06 02:00:00,desktop,1,False,105,2.0,NaN,2021-09-06
7,2021-09-13 02:00:00,desktop,1,False,131,2.0,26.0,2021-09-13
8,2021-09-20 02:00:00,desktop,1,False,142,2.0,11.0,2021-09-20
10,2021-09-13 03:00:00,desktop,1,False,113,3.0,-4.0,2021-09-13
...,...,...,...,...,...,...,...,...
990,2021-09-05 18:00:00,touch,7,True,2098,18.0,NaN,2021-09-05
993,2021-09-05 19:00:00,touch,7,True,2047,19.0,NaN,2021-09-05
998,2021-09-19 20:00:00,touch,7,True,2371,20.0,-72.0,2021-09-19
1003,2021-09-12 22:00:00,touch,7,True,1985,22.0,154.0,2021-09-12
